In [1]:
# import libraries
from pyspark import SparkContext
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
appName="Collaborative Filtering with PySpark"
# initialize the spark session
spark = SparkSession.builder.appName(appName).getOrCreate()
# get sparkcontext from the sparksession
sc = spark.sparkContext

In [4]:
#define schema
schema = StructType([
    StructField("item", StringType(), True),
    StructField("user", StringType(), True),
    StructField("rating", StringType(), True),
    StructField("timestamp", IntegerType(), True)])
#read the file as a dataframe
df = spark.read.csv("Gift_Cards.csv",header=False,schema=schema)
#print the schema
df.printSchema()
#show the dataframe header
#df.show(n=5)
#number of rows
df.count()
#convert rating colum from string to integer
df = df.withColumn("rating", df["rating"].cast(IntegerType()))
df.show(n=5)
#provide index values for item and user to convert them into integers
stringIndexer = StringIndexer(inputCols=["item","user"], outputCols=["itemIndex","userIndex"])
model = stringIndexer.fit(df)
df_indexed = model.transform(df)
df_indexed.show(n=5)

root
 |-- item: string (nullable = true)
 |-- user: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: integer (nullable = true)

+----------+--------------+------+----------+
|      item|          user|rating| timestamp|
+----------+--------------+------+----------+
|B001GXRQW0| APV13CM0919JD|     1|1229644800|
|B001GXRQW0|A3G8U1G1V082SN|     5|1229472000|
|B001GXRQW0| A11T2Q0EVTUWP|     5|1229472000|
|B001GXRQW0| A9YKGBH3SV22C|     5|1229472000|
|B001GXRQW0|A34WZIHVF3OKOL|     1|1229472000|
+----------+--------------+------+----------+
only showing top 5 rows

+----------+--------------+------+----------+---------+---------+
|      item|          user|rating| timestamp|itemIndex|userIndex|
+----------+--------------+------+----------+---------+---------+
|B001GXRQW0| APV13CM0919JD|     1|1229644800|     39.0| 120184.0|
|B001GXRQW0|A3G8U1G1V082SN|     5|1229472000|     39.0|  87652.0|
|B001GXRQW0| A11T2Q0EVTUWP|     5|1229472000|     39.0|  13165.0|
|B001GX

In [5]:
#split the data into training and testing set
(training, test) = df_indexed.randomSplit([0.8, 0.2])
#training the model
#define the model parameters
als = ALS(maxIter=5, 
          implicitPrefs=False,
          userCol="userIndex", 
          itemCol="itemIndex", 
          ratingCol="rating",
          coldStartStrategy="drop")
#train the model
model = als.fit(training)

In [6]:
# predict using the testing datatset
predictions = model.transform(test)
predictions.show()

+----------+--------------+------+----------+---------+---------+-----------+
|      item|          user|rating| timestamp|itemIndex|userIndex| prediction|
+----------+--------------+------+----------+---------+---------+-----------+
|B00MV9O08G| A8E9XARLHNQRB|     5|1461196800|    148.0|   9852.0|  1.8560327|
|B00MV9O08G|A1I947AYYVP521|     5|1454716800|    148.0|   4477.0|-0.34030437|
|B00MV9O08G| ABKWU9EIM7C9B|     5|1466035200|    148.0|  10053.0|-0.14910448|
|B00MV9O08G|A269V5L0P4YLA6|     5|1468627200|    148.0|    493.0| -3.1651788|
|B00MV9O08G|A3JVYMD4HETANN|     5|1485734400|    148.0|    142.0| -4.6776433|
|B00MV9O08G|A2OSD7R4SAF15Y|     5|1508457600|    148.0|   1028.0|   1.032892|
|B00MV9O08G|A34CF62DEN31Q9|     5|1478304000|    148.0|    324.0|  0.9895096|
|B00MV9O08G|A241GI5D9EVJI6|     5|1482278400|    148.0|    482.0|  1.0802039|
|B00MV9O08G| A6VLDZ3EA1YW0|     5|1474329600|    148.0|   2981.0|  1.6228987|
|B00MV9O08G| AA47T2SDZBR8A|     5|1487808000|    148.0|   9960.0

In [8]:
def topLikes(dataframe,userIndex,limit):
 df = dataframe.filter(dataframe.userIndex==userIndex)\
 .sort(dataframe.rating.desc())\
 .select(dataframe.userIndex,dataframe.itemIndex,dataframe.rating)\
 .limit(limit)
 return df
# display top liked items for a user
topLikes(df_indexed,9386,10).show(truncate=False)

+---------+---------+------+
|userIndex|itemIndex|rating|
+---------+---------+------+
|9386.0   |463.0    |5     |
|9386.0   |53.0     |4     |
+---------+---------+------+



In [13]:
def recommendedItems(userIndex,limit):
    test =  model.recommendForAllUsers(3)\
        .filter(col('userIndex')==userIndex)\
        .select(["recommendations.itemIndex","recommendations.rating"])\
        .collect()
    return test
# display top 10 recommended artists for user 2062243
recommendedItems(9386,10)

[Row(itemIndex=[1208, 1016, 1417], rating=[8.179315567016602, 7.899316310882568, 7.583199977874756])]